# Utils

In [1]:
import os
import glob
import librosa
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import train_test_split


# Loading Dataset

In [4]:
# Verzeichnispfad anpassen
audio_dir = "data"
# Liste aller Audiodateien
filepaths = glob.glob(os.path.join(audio_dir, '**/*.wav'), recursive=True)

# Daten und Labels speichern
data = []
labels = []

for filepath in filepaths:
  # Audio laden und in NumPy-Array konvertieren
  y, sr = librosa.load(filepath, sr=None)

  # Label aus Dateiname extrahieren (z.B. "knock_001.wav" -> 1)
  label =  1 if os.path.basename(filepath).split("_")[0] == "knock" else 0

  # Daten und Labels hinzufügen
  data.append(y)
  labels.append(label)


## testing loaded Data

In [10]:
print(data[0], labels[0])

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
# Daten in Tensoren konvertieren
data_tensor = torch.from_numpy(np.array(data)).float()

import torchvision.transforms as transforms

# Define the mean and standard deviation for normalization
mean = data_tensor.mean()
std = data_tensor.std()

np.save('data/npy/mean.npy', mean)
np.save('data/npy/std.npy', std)

# Create the transform
transform = transforms.Normalize(mean=mean, std=std)

# Apply the transform to the data_tensor
normalized_data = transform(data_tensor)

labels_tensor = torch.from_numpy(np.array(labels)).long()

# Teilen der Daten in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(data_tensor, labels_tensor, test_size=0.2)


# Training Model

In [ ]:

class AudioClassifier(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(AudioClassifier, self).__init__()
    self.lstm = nn.LSTM(input_size, hidden_size)
    self.fc = nn.Linear(hidden_size, num_classes)

  def forward(self, x):
    # x: (batch_size, sequence_length, input_size)
    x, _ = self.lstm(x)
    # x: (batch_size, sequence_length, hidden_size)
    out = self.fc(x[:, -1, :])  # Zugriff auf den letzten Output des LSTMs
    # out: (batch_size, num_classes)
    return F.log_softmax(out, dim=1)  # Log-Softmax für Klassifikation

# Hyperparameter
input_size = 13  # Anzahl Features (z.B. aus MFCCs)
hidden_size = 64
num_classes = 2  # Klopfen (0) und Geräusche (1)

# Modell initialisieren
model = AudioClassifier(input_size, hidden_size, num_classes)

# Optimierer und Verlustfunktion definieren
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.NLLLoss()

# Trainingsdaten laden (muss angepasst werden)
# ...

# Trainieren des Modells
for epoch in range(10):
  # Trainings-Schleife (muss angepasst werden)
  # ...

  # Validierung des Modells (optional)
  # ...
    break
# Speichern des Modells
  
torch.save(model.state_dict(), "audio_classifier.pt")
